### FORECAST DATA

In [ ]:
def append_data(current_data, additional_data):

    current_data = current_data.reset_index()
    additional_data = additional_data.reset_index()

    current_data = pd.concat([current_data[current_data['datetime'].isin(
        additional_data['datetime']) == False], additional_data], ignore_index=True)

    # print(data)
    current_data.dropna(subset=['datetime'], inplace=True)
    current_data.sort_values(by=['datetime'], inplace=True)

    current_data = current_data.set_index('datetime')
    current_data = current_data[current_data.index.notnull()]

    return current_data

In [ ]:
output_path = wdir + '_output_fct_/'
output_dt_format = '%Y-%m-%d %H:%M:%S'

In [ ]:
# stations metadata
metadata_data_path = wdir + "/data/forecast/points3.csv"

metadata = pd.read_csv(metadata_data_path)
# metadata = metadata[ metadata['ID_UI'] != '-999' ]


In [ ]:
############## FORECAST DATA ##############
for variable in variables:
    input_fct_data = wdir + 'data/forecast/icon-d2-eps_45h/postprocessed/' + \
        basin + '/' + variable + '/deterministic/mean/'

    if variable == 'precipitation':
        precipitation_fct_table = pd.DataFrame(index=range(
            1, forecasting_lead_time+1), columns=precipitation_data.columns)
    elif variable == 'temperature':
        temperature_fct_table = pd.DataFrame(index=range(
            1, forecasting_lead_time+1), columns=temperature_data.columns)

    dates = glob.glob(input_fct_data + '/*')

    for date in dates:

        date_str = date.split('/')[-1]
        # print(date_str)

        files = glob.glob(date + '/*.csv')

        for f in files:
            id_point = str(os.path.basename(f)[:-4])
            # print(id_point)

            # ID,2.0
            # x,642993.5
            # y,5164882.0
            # z,630.0
            header = """ID,{id}\nx,{x}\ny,{y}\nz,{z}\n"""

            df = read_timeseries_pd(pd.read_csv(
                f, sep=';'), '%Y-%m-%d %H:%M:%S')
            df = df.rename(columns={'values': id_point})
            df = df.iloc[-forecasting_lead_time:]

            try:
                if variable == 'precipitation':
                    obs_data = precipitation_data[[id_point]]
                elif variable == 'temperature':
                    obs_data = temperature_data[[id_point]]
            except:
                continue
            current_data = append_data(obs_data, df)

            # current_data = current_data.resample('H').mean()
            current_data.index = [dt.datetime.strftime(
                i, format=output_dt_format) for i in current_data.index]
            current_data.index.name = 'datetime'

            df = current_data.to_csv(header=False).strip('\n').split('\n')
            data = '\r\n'.join(df)

            output_filepath = output_path + date_str + \
                "/" + variable + "/" + id_point + ".csv"
            mkNestedDir(getPathFromFilepath(output_filepath))

            try:
                east = float(metadata[metadata['ID'] == int(id_point)]['lat'])
                north = float(metadata[metadata['ID'] == int(id_point)]['lon'])
                elevation = float(
                    metadata[metadata['ID'] == int(id_point)]['elev'])
            except:
                print('No metadata: ' + id_point)
                raise

            with open(output_filepath, 'w') as new:
                new.write(header.format(id=id_point,
                                        x=east, y=north, z=elevation))
                new.write(data)

## TODO > precipitation_fct_table must be used!